In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier

In [19]:
# -------------------data import----------------------------------------------------------
# https://archive.ics.uci.edu/ml/datasets/heart+Disease
# Cleveland dataset
data_cl = pd.read_csv('../datasets/processed.cleveland_2.txt',\
                      sep=",", header=None) 
# Hungarian dataset
data_hu = pd.read_csv('../datasets/processed.hungarian_2.txt',\
                      sep=",", header=None) 
# Swiz dataset
data_sw = pd.read_csv('../datasets/processed.switzerland_2.txt',\
                      sep=",", header=None)
# Long beach dataset
data_va = pd.read_csv('../datasets/processed.va_2.txt',\
                      sep=",", header=None)

# Merge all datasets into one dataset
data = pd.concat([data_cl,data_hu,data_sw,data_va]).replace('?', np.nan).reset_index(drop=True)
# Column names, according to the web source
data.columns = ['age', 'sex','cp','trestbps','chol','fbs','restecg',\
                'thalach','exang','oldpeak','slope','ca','thal','num']

# Filter out datapoints with missing values
filtered_data = data[data['trestbps'].notnull()]
filtered_data = filtered_data[filtered_data['chol'].notnull()]
filtered_data = filtered_data[filtered_data['age'].notnull()]
filtered_data = filtered_data[filtered_data['sex'].notnull()]
filtered_data = filtered_data[filtered_data['restecg'].notnull()]
filtered_data = filtered_data.drop(filtered_data[filtered_data.chol == 0].index).reset_index(drop=True)
filtered_data = filtered_data.drop(filtered_data[filtered_data.chol == '0'].index).reset_index(drop=True)

# Make label class into binary
filtered_data['num'] = filtered_data['num'].astype(bool).astype(int).values.tolist()
# Make relevant columns into list values
filtered_data['chol'] = filtered_data['chol'].astype(int)
filtered_data['trestbps'] = filtered_data['trestbps'].astype(int)
filtered_data['age'] = filtered_data['age'].astype(int)
filtered_data['sex'] = filtered_data['sex'].astype(int)
filtered_data['restecg'] = filtered_data['restecg'].astype(int)


# Divide into two subsets (sick and healthy) for data generation
sick = filtered_data.loc[filtered_data['num'] == 1]
good = filtered_data.loc[filtered_data['num'] == 0]

# Compute avg and var for the sensor values
avg_chol_sick = np.mean(sick['chol'].astype(np.float).values.tolist())
std_chol_sick =  np.std(sick['chol'].astype(np.float).values.tolist())
avg_chol_good = np.mean(good['chol'].astype(np.float).values.tolist())
std_chol_good = np.std(good['chol'].astype(np.float).values.tolist())

avg_bp_sick = np.mean(sick['trestbps'].astype(np.float).values.tolist())
std_bp_sick = np.std(sick['trestbps'].astype(np.float).values.tolist())
avg_bp_good = np.mean(good['trestbps'].astype(np.float).values.tolist())
std_bp_good = np.std(good['trestbps'].astype(np.float).values.tolist())

label = filtered_data.num.astype(bool).astype(int).values.tolist() # binary classes, heart disease or not
age = filtered_data.age.values.tolist()
bp = filtered_data.trestbps.values.tolist() #blood pressure
chol = filtered_data.chol.values.tolist() # cholesterol
ecg = filtered_data.restecg.values.tolist() #electrocardiography

sex = filtered_data.sex.values.tolist()
bloodsugar = filtered_data.fbs.values.tolist() # fasting bloodsugar
maxhr = filtered_data.thalach.values.tolist() # max heart rate
exang = filtered_data.exang.astype(int).values.tolist() # exercise induced angia
oldpeak = filtered_data.oldpeak.values.tolist() # ST depression induced by exercise relative to rest
slope = filtered_data.slope.values.tolist() # the slope of the peak exercise ST segment, 1: upsloping, 2: flat, 3: downsloping
ca = filtered_data.ca.values.tolist() # number of major vessels (0-3) colored by flourosopy
thal = filtered_data.thal.values.tolist() #3 = normal; 6 = fixed defect; 7 = reversable defect

In [41]:
from sklearn.gaussian_process.kernels import  WhiteKernel, RBF, DotProduct, ExpSineSquared, Matern
from sklearn.gaussian_process.kernels import RationalQuadratic, Sum, Product, ConstantKernel

In [47]:
#Kernel exploring
x = np.transpose([age,sex,chol,bp,ecg,exang])

kernel = 1*RBF( length_scale =1) + WhiteKernel ( noise_level =0.1)
gpc = GaussianProcessClassifier(kernel=kernel)

kernels1 = [WhiteKernel(noise_level =0.1) + 2**1*RBF(length_scale =1), \
           WhiteKernel(noise_level =0.1) + 2**1*DotProduct(), \
           WhiteKernel(noise_level =0.1) + 2**1*Matern()]
kernelDescription1 = ["White and RBF","White and DotProduct","White and Matern"]

kernels2 = [WhiteKernel(noise_level =0.1) + 2**1*RationalQuadratic(), \
           WhiteKernel(noise_level =0.1) + ConstantKernel(), \
           WhiteKernel(noise_level =0.1) + 2**1*Product(RBF(),DotProduct()),\
           WhiteKernel(noise_level =0.1) + 2**1*Sum(RBF(),DotProduct())]
kernelDescription2 = ["White and RQ","White and Constant","White and product of RBF and DotProduct","White and sum of RBF and DotProduct"]

#kernels = kernels2
#kernelDescription = kernelDescription2

kernels = kernels1 + kernels2 
kernelDescription = kernelDescription1 + kernelDescription2

splits = 5
kernelIndex = 0

AvBestScore = 0
BestKernelIndex = 0

for kernel in kernels:
    
    gpc = GaussianProcessClassifier(kernel=kernel)
    sc = cross_validate(gpc, x, y.ravel(), cv=splits)
    
    if(np.mean(sc["test_score"]) > AvBestScore):
        AvBestScore = np.mean(sc["test_score"])
        BestKernelIndex = kernelIndex
    
    print("------------Kernel info--------------")
    print("Kernel: ", kernelDescription[kernelIndex])
    print("Average score: ", np.mean(sc["test_score"]))
    print("Best score: ", np.max(sc["test_score"]))
    print("Worst score: ", np.min(sc["test_score"]))
    print("-------------------------------------")
    
    kernelIndex = kernelIndex + 1

print("------------Result---------------")
print("Best kernel: ", kernelDescription[BestKernelIndex])
print("Best average score: ", AvBestScore)
print("---------------------------------")
    

------------Kernel info--------------
Kernel:  White and RBF
Average score:  0.6496899489480701
Best score:  0.746268656716418
Worst score:  0.5223880597014925
-------------------------------------
------------Kernel info--------------
Kernel:  White and DotProduct
Average score:  0.7568542906383117
Best score:  0.8656716417910447
Worst score:  0.7111111111111111
-------------------------------------
------------Kernel info--------------
Kernel:  White and Matern
Average score:  0.5993057587877606
Best score:  0.7037037037037037
Worst score:  0.5447761194029851
-------------------------------------
------------Kernel info--------------
Kernel:  White and RQ
Average score:  0.7048806620492309
Best score:  0.7761194029850746
Worst score:  0.5481481481481482
-------------------------------------
------------Kernel info--------------
Kernel:  White and Constant
Average score:  0.5281930218190095
Best score:  0.5298507462686567
Worst score:  0.5259259259259259
------------------------------

In [48]:
#Visualiation
#x = np.transpose([age,sex,chol,bp,ecg,exang])

gpc = GaussianProcessClassifier(kernel=kernels[BestKernelIndex]).fit(x,y.ravel())

In [ ]:
plt.figure(figsize=(3 * 2, n_classifiers * 2))
plt.subplots_adjust(bottom=.2, top=.95)

x = bp
y = chol

xx = np.linspace(np.min(x)-10, np.max(x)+10, 100)
yy = np.linspace(np.min(y)-10, np.max(y)+10, 100).T
xx, yy = np.meshgrid(xx, yy)
Xfull = np.c_[xx.ravel(), yy.ravel()]

